In [ ]:
# %pip install pymssql

In [1]:
import pymssql
import pandas as pd
import redis
import time
import datetime
from ipywidgets import IntProgress

In [2]:
#REDIS_IP = '10.2.5.212'
REDIS_IP = 'localhost'

In [3]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
            autocommit=True
        )

In [4]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [5]:
def concatenate_linkedid_side(side, record_date, linkedid):
    query = "SELECT text from transcribations where "
    query += " side="+str(side)+" and "
    query += " record_date = '"+str(record_date)+"' and "
    query += " linkedid = '"+str(linkedid)+"';"
    text_df = read_sql(query)
    phrases_count = len(text_df)
    text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
    return text_full, phrases_count

In [37]:
def summarize(text, phrases_count):
    #if phrases_count<2 or len(text)<255:
    #    return text
    subscriber = redis.StrictRedis(host=REDIS_IP)
    publisher = redis.StrictRedis(host=REDIS_IP) 
    pub = publisher.pubsub()
    sub = subscriber.pubsub()
    sub.subscribe('summarus_client')
    # send
    publisher.publish("summarus_server", text)
    # receive
    while True:
        message = sub.get_message()
        if message and message['type']!='subscribe':
            return message['data'].decode("utf-8")
        time.sleep(1)

In [ ]:
def sum_to_sql(linkedid, recor_date, side, text, phrases_count, text_length):
    current_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    query = "insert into summarization(linkedid, record_date, sum_date, side, text, phrases_count, text_length) "
    query += " values("
    query += "'"+str(linkedid)+"',"
    query += "'"+str(recor_date)+"',"
    query += "'"+str(current_date)+"',"
    query += str(side)+","
    query += "'"+str(text)+"',"
    query += "'"+str(phrases_count)+"',"
    query += "'"+str(text_length)+"'"
    query += ");"
    #df = read_sql(query)
    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)
    # conn.commit()

In [ ]:
# queue
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
queue = read_sql(query)
queue

In [ ]:
# transcribations
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
queue = read_sql(query)
queue

### queue linkedid list

In [ ]:
# queue
query = "SELECT distinct record_date, linkedid from queue;"
queue = read_sql(query)
len(queue)

### transcribations linkedid list

In [ ]:
# queue
query = "SELECT distinct top 10 record_date, linkedid from transcribations;"
df = read_sql(query)
df

In [ ]:
query = "SELECT distinct top 100 trans.record_date, trans.linkedid"
query += " from transcribations as trans"
query += " left join queue as queue"
query += " on trans.record_date=queue.record_date"
query += " and trans.linkedid=queue.linkedid"
query += " where "
query += " trans.record_date > '2021-06-04 00:00:00' and"
query += " isnull(queue.linkedid,'x')='x'"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

# start

In [7]:
BATCH_SIZE = 1000
MAX_TEXT_SIZE = 1023

In [ ]:
# obtain datetime limits
#query = "select min(record_date) from queue where not isnull(record_date,'2021-06-07 13:34:51')='2021-06-07 13:34:51';"
query = "select linkedid from queue;"
df = read_sql(query)
if len(df):
    query = "select min(record_date) from queue where not isnull(record_date,'')='';"
    df = read_sql(query)
    queue_first_record = str(df.iloc()[0][0])
else:
    queue_first_record = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

query = "select max(record_date) from summarization;"
df = read_sql(query)
summarization_first_record = str(df.iloc()[0][0])

In [ ]:
queue_first_record, summarization_first_record

In [ ]:
query

In [ ]:
query = "SELECT distinct top "+str(BATCH_SIZE)+" record_date, linkedid"
query += " from transcribations"
query += " where "
query += " record_date < '"+queue_first_record+"' and"
query += " not linkedid in (select distinct linkedid from summarization)"
query += " order by record_date desc;"
df = read_sql(query)
len(df)

In [ ]:
#text_full

In [ ]:
for _id, row in df.iterrows():
    
    for side in range(2):
        text_full, phrases_count = concatenate_linkedid_side(side, row.record_date, row.linkedid)
        text_short = summarize(text_full, phrases_count)
        sum_to_sql(row.linkedid, row.record_date, side, text_short, phrases_count, len(text_full))

In [ ]:
len('Российский мастер по ремонту стиральных машин, с кем можно переговорить, пожаловался на то, что белье не попадает в барабан, барабан не крутится, не вращается, он перетащил машинку с одного места на второе, а она почему-то перестала стирать, да, да нет москва, а я давно как ты взял эту самую визитку у этого почтовом ящике и вот по ней и звоню айсберг вот визитка, да конечно номер триста шестнадцать серия, спасибо спасибо, да пожалуй, да.')

In [ ]:
query = "SELECT * from summarization"
query += " where  record_date = '2021.05.30 11:46:10' and linkedid = '1622364369.119129' and side = '0';"
df = read_sql(query)
df

In [ ]:
def summarization():
    message = ''
    query = "SELECT count(linkedid) from summarization"
    query += " where isnull(load_msk,0) = 0 or isnull(load_spb,0) = 0 or isnull(load_reg,0) = 0;"
    df = read_sql(query)
    message += '\nСуммаризации, ожидающие загрузку: ' + str(df.iloc()[0][0])
    
    message += '\n== За вчера =='
    
    currentdate = datetime.datetime.today()
    start_of_day = currentdate.combine(currentdate.date(), currentdate.min.time())
    yesterday = start_of_day + datetime.timedelta(days=-1)
    df = str(yesterday)
    dt = str(start_of_day)
    f_0 = '%Y-%m-%d %H:%M:%S'
    f_1 = '%Y-%m-%dT%H:%M:%S'
    date_from = datetime.datetime.strptime(df, f_0).strftime(f_1)
    date_toto = datetime.datetime.strptime(dt, f_0).strftime(f_1)
    
    query = "SELECT count(linkedid) from summarization"
    query += " where sum_date>'" + date_from + "'"
    query += " and sum_date<'" + date_toto + "';"
    df = read_sql(query)
    message += '\nНовых суммаризаций: ' + str(df.iloc()[0][0])
    
    for city in ['load_msk', 'load_spb', 'load_reg']:
        query = "SELECT count(linkedid) from summarization"
        query += " where sum_date>'" + date_from + "'"
        query += " and sum_date<'" + date_toto + "'"
        query += " and isnull("+city+",0) = 1;"
        df = read_sql(query)
        message += '\n'+city+': ' + str(df.iloc()[0][0])      
    
    return message
    
summarization()

In [77]:
query = "SELECT count(linkedid) from summarization;"
df = read_sql(query)
df.iloc()[0][0]

145134

In [78]:
query = "SELECT count(linkedid) from summarization;"
df = read_sql(query)
df.iloc()[0][0]

145137

In [ ]:
query = "SELECT max(sum_date) from summarization;"
df = read_sql(query)
df

In [ ]:
query = "SELECT * from summarization order by sum_date desc;"
df = read_sql(query)
min(df.record_date), max(df.record_date)

In [ ]:
df

In [ ]:
query = "SELECT * from summarization order by sum_date desc;"
df = read_sql(query)
#for _id, row in df.iterrows():
    #df.text.iloc()[3]
#    print(row.sum_date, row.record_date)
min(df.record_date), max(df.record_date)

In [ ]:
query = "SELECT max(record_date) from transcribations;"
df = read_sql(query)
df

In [ ]:
min(df.record_date), max(df.record_date)

In [ ]:
min(df.record_date), max(df.record_date)

In [ ]:
row

query = "SELECT column_name FROM information_schema.columns WHERE table_name='summarization';"
queue = read_sql(query)
queue

In [ ]:
query = "SELECT linkedid, side, text from summarization where "
query += " linkedid = '1622976650.277074';"
df = read_sql(query)
df

In [ ]:
# select
query = "SELECT side, text from transcribations where "
query += " record_date = '2021-06-04 12:33:20' and linkedid = '1622799198.236534';"
df = read_sql(query)
', '.join([row.text for _id, row in df.iterrows()])

In [ ]:
len('здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо, и, кузбасс сервиса оператору, могу принять заявку в течение часа мастер перезвонит у него все уточнить и подберёте временно визит живете в москве и области, холодильник как называется, гарантия закончилась от производителя холодильник двухкамерные трёхкамерные что не работает, и, промерзание шкафа да да, вы нас стефан узнали где магнит визитка сервисная книжка, вверху справа три цифры какие стоят на магнитных карт сможет отказать, на, спасибо мы указываем адрес телефон в течение час поступит звонок телефон вас скажите, зовут вас как, гален улиц какая у вас, номер дома, корпус есть, квартира, подъезд и этаж, метро какое-то ближайшее, давайте проверим а несколько тридцать пять корпус два квартира два пять семь подъезд третий этаж пятый телефон девятьсот девять девять восемь один пять один семьдесят северном, техника авто номер вашей заявки будет сто пятьдесят девять сорок пять восемьдесят четыре, сорок пять восемьдесят четыре ждите звонок пожалуйста, пожалуйста свидания')

In [ ]:
a = 'hello world'
a[:3]

In [ ]:
#df.text.iloc()[3]

# DELETE

In [ ]:
# delete
query = "DELETE from summarization;"
conn = ms_sql_con()  
#cursor = conn.cursor()
#cursor.execute(query)

In [ ]:
tx = "Statesman's Dilemma (Стандард) (США) (английский) (на английском языке) (в переводе на русский) (с английской буквы «С» — «С») (СССР) (далее — США) (Атлант) (КНР) (Англия) (Британия) (Нью-Йорк) (Великобритания, Канада) (Боснийская) (Монголия)"

In [ ]:
tx.replace("'","").replace('"','')

In [ ]:
side = 0
query = "SELECT text, source_id from transcribations where "
query += " side="+str(side)+" and "
#query += " record_date = '"+str(record_date)+"' and "
query += " linkedid = '1622799198.236534';"
text_df = read_sql(query)
phrases_count = len(text_df)
text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
text_full, phrases_count, min(text_df.source_id)

In [ ]:
# update summarization field

In [ ]:
query = "SELECT distinct"
query += " summarization.record_date, summarization.linkedid, transcribations.source_id"
query += " from summarization"
query += " inner join transcribations on"
query += " summarization.record_date = transcribations.record_date and"
query += " summarization.linkedid = transcribations.linkedid"
query += " where "
query += " isnull(summarization.source_id,-1)=-1;"
df = read_sql(query)
len(df)

In [ ]:
# 13518

In [ ]:
prgBar = IntProgress(min = 0, max = len(df))
display(prgBar)
for _id, row in df.iterrows():
    query = "update summarization set source_id = '"+str(row.source_id)+"' where "
    query += " record_date = '"+str(row.record_date)+"' and"
    query += " linkedid = '"+str(row.linkedid)+"';"    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)   
    prgBar.value = prgBar.value + 1

In [ ]:
query = "SELECT * from summarization where "
query += " load_msk = 1;"
df = read_sql(query)
len(df)

### quality upgrade

In [113]:
linkedid = '1622701911.206736'

In [114]:
query = "SELECT top 1 record_date from transcribations where "
query += " linkedid = '"+linkedid+"';"
df = read_sql(query)
record_date = df.record_date.iloc()[0]
record_date

Timestamp('2021-06-03 09:31:52')

In [115]:
text_full, phrases_count = concatenate_linkedid_side(
    side = 1, 
    record_date = record_date,
    linkedid = linkedid
)
text_full

'и это нас наш рассматр накалять расстаться, да, да пожалуйста как называется машина, она не на гарантии, оставь случилось, в москве находится, я могу предложить вызвать мастера на сегодня сейчас составляете за да оставляйте заявку мастера звонят течение двух часов вы с ним всё обговаривается и назначайте время визита, записываем а где вы увидели наш номер телефона, да, ясно как вас зовут, адрес пожалуйста, так, подъезд и этаж, код домофон, но вот телефона, да какой вам удобнее, так, метро ближайшая, спасибо давайте проверим москва подъёмная один квартира двести пятнадцать подвес первая та с тринадцатой телефон девятьсот три двести один ноль семь двадцать восемь, не надо спасибо это я для компьютера номер вашего заказа сто пятьдесят девять, тридцать девять шестьдесят один, да нет это не столь важно все а самое главное ожидать звонок мастера, спасибо до свидания'

In [116]:
text_short = summarize(text_full, phrases_count)
text_short

'В москве произошла авария, в результате которой погибли три человека, в том числе и я.'

In [117]:
text_short = summarize(text_full.replace(',',' '), phrases_count)
text_short

'В москве произошла авария, в которой погибли два человека, в том числе и я, и двое детей, в результате чего один из них был госпитализирован с серьезными травмами.'

In [118]:
text_short = summarize(text_full.replace(',',' - '), phrases_count)
text_short

'В москве у нас произошла авария - машина не на гарантии - мастера не приехали.'

In [86]:
### stage 2

In [26]:
text_dropped = ''
drop_words = ['один',
              'два',
              'четыр',
              'пять',
              'пятнад',
              'шесть',
              'семь',
              'восем',
              'девят',
              'десят',
              'одинадцат',
              'двенадцат',
              'тринадцат',
              'шестнад',
              'семнад',
              'тридцат',
              'сорок',
              'девяност',
              'сто',
              'двест',
              'трист'
             ]
text_dropped = ''
for word in text_full.split(' '):
    word_ok = True
    for drop in drop_words:
        if drop in word:
            word_ok = False
            break
    if word_ok:
        text_dropped += word+' '
text_dropped

'и единая служба сервиса оператор ноэля оставаться, день дом, основном, да у аверченко одном он закреплён заявка, даже не смотрел совершенно крепится татьяна так проверяем беловежская квартира телефона и городской новый номер заявки запишите повторно по гарантии на ремонт после ожидаете звонок мастера цыплята права '

In [32]:
text_dropped[:int(len(text_dropped)/2)], text_dropped[int(len(text_dropped)/2):]

('и единая служба сервиса оператор ноэля оставаться, день дом, основном, да у аверченко одном он закреплён заявка, даже не смотрел совершенно крепится татьяна т',
 'ак проверяем беловежская квартира телефона и городской новый номер заявки запишите повторно по гарантии на ремонт после ожидаете звонок мастера цыплята права ')

In [49]:
text_short = summarize(text_full, phrases_count)
text_short

'В Беловежской области произошла авария, в результате которой погибли пять человек, в том числе двое детей.'

In [51]:
wrong_words = ['погиб']
for wrong in wrong_words:
    if wrong in text_short:
        print('y')
'k'

y


'k'

In [56]:
'в р' in text_short

True

In [48]:
text_short = summarize(text_full, phrases_count)

wrong_words = ['погиб']
for wrong in wrong_words:
    if wrong in text_short:
        text_short = summarize(text_full.replace(',',''), phrases_count)
        break

for wrong in wrong_words:
    if wrong in text_short:
        text_short = text_full[:1023]
        break
text_short

'Эксперты считают, что ремонтные работы в московских квартирах, в том числе и в новостройках, должны быть выполнены в соответствии с запланированными сроками, а не по инициативе владельцев.'

In [46]:
text_full[:10]

'и единая с'

In [41]:
text_short = summarize(text_full.replace(',',''), phrases_count)
text_short

'Эксперты считают, что ремонтные работы в московских квартирах, в том числе и в новостройках, должны быть выполнены в соответствии с запланированными сроками, а не по инициативе владельцев.'

In [39]:
text_short = summarize(text_dropped[:int(len(text_dropped)/2)].replace(',', ''), phrases_count)
text_short

'В ноябре прошлого года оператор ноэля работал в Санкт-Петербурге и Ленинградской области, а также в Москве и Санкт-Петербургской области.'

# reports

In [64]:
query = "SELECT * from summarization where sum_date>'2021.06.18 09:19:50' and not text='' order by sum_date desc;"
df = read_sql(query)
for id, row in df.iterrows():
    print(row.text)

и
Скажите пожалуйста, у меня плита в квартире не индезит из-за того, что я не успела сделать диагностику.
На днях у нас произошла неисправность газовой плиты у нас был мастер, который чинил конфорку само, он сказал, что надо заказывать, но я не знаю, сколько это будет стоить.
и, мама, да а вы откуда с какой шотландка, лазер кэш уже отменял, но я буду новость в том что передумал, да ел я уже нового буду на образном на эти закрыть рот запёкся, да я уже позвонил к вам, служба уже отменил заказ, да да спасибо
алло а это значит холодильника мастер, это альберт, а почему опьянели, хотели новых купите новых, а ты чётко, так все решила сосну
и
и, и, э, э, э, и, и
и, и, э
Российский оператор наталья роста цен, который работает на московском рынке холодильников, рассказал, что с ним случилось.
Российские мастера по ремонту холодильника, бош, бытовой техники, которые работают в Москве и Подмосковье, предлагают свои услуги.
и
и
и
и, здравствуйте
и
и, алло, но привозите я думаю сами доставим, да
St

In [75]:
query = "SELECT * from summarization where text like '%погиб%' order by sum_date desc;"
df = read_sql(query)
for id, row in df.iterrows():
    print(row.text)

In [76]:
len(df)

0

In [74]:
#query = "delete from summarization where text like '%погиб%';"
#df = read_sql(query)